In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#show more columns
pd.set_option('display.max_columns', None)

In [121]:
data = pd.read_csv('data/IBD NSQIP Year 5 merged v1.csv',na_values='Unknown')
print(data.shape)
#data.head()

(1339, 236)


In [122]:
list(data.columns)

['case.number',
 'completion.status',
 'date.of.birth',
 'age.at.time.of.surgery',
 'gender',
 'race',
 'hispanic.ethnicity',
 'cpt.code',
 'cpt.description',
 'principal.operative.procedure',
 'in.out.patient.status',
 'elective.surgery',
 'origin.status',
 'hospital.admission.date',
 'operation.date',
 'principal.anesthesia.technique',
 'additional.anesthesia.technique.s.',
 'surgical.specialty',
 'height',
 'height.unit',
 'height.unknown',
 'weight',
 'weight.unit',
 'weight.unknown',
 'bmi',
 'diabetes.mellitus',
 'current.smoker.within.1.year',
 'dyspnea',
 'functional.heath.status',
 'ventilator.dependent',
 'history.of.severe.copd',
 'ascites.w.in.30.days.prior.to.surgery',
 'congestive.heart.failure.w.in.30.days.prior.to.surgery',
 'hypertension.requiring.medication',
 'acute.renal.failure',
 'currently.requiring.or.on.dialysis',
 'disseminated.cancer',
 'open.wound.with.or.without.infection',
 'steroid.immunosuppressant.use.for.chronic.condition',
 'malnourishment',
 'bleedin

In [123]:
#subset to only CPT code 44205 or 44160
data = data.loc[data['cpt.code'].isin([44205,44160]) | data['cpt.code'].isin(['44205','44160'])]
print(data.shape)
#drop all observations that have 'ileostomy.formation' set to missing
data = data.dropna(subset=['ileostomy.formation'])
print(data.shape)


(374, 236)
(280, 236)


In [124]:
data['colectomy.primary.indication.for.surgery'].value_counts()

Crohn's Disease                     256
Colon cancer                          8
Ulcerative colitis                    5
Other-Enter ICD-10 for diagnosis      5
Chronic diverticular disease          4
Colon cancer w/ obstruction           1
Non-malignant polyp                   1
Name: colectomy.primary.indication.for.surgery, dtype: int64

In [125]:
#Only want Crohn's disease
data = data.loc[data['colectomy.primary.indication.for.surgery'].isin(["Crohn's Disease"])]
print(data.shape)


(256, 236)


In [126]:
#combine colectomy.operative.approach categories
data['colectomy.operative.approach'] = data['colectomy.operative.approach'].replace({'Endoscopic w/ unplanned conversion to open':'MIS unplanned conversion to open','Hybrid':'MIS','Laparoscopic':'MIS','Laparoscopic w/ open assist':'MIS','Laparoscopic w/ unplanned conversion to open':'MIS, unplanned conversion to open','Open (planned)':'Open','Robotic':'MIS','Robotic w/ open assist':'MIS','Robotic w/ unplanned conversion to open':'MIS, unplanned conversion to open','SILS':'MIS','SILS w/ unplanned conversion to open':'MIS'})
#binarize 'ileostomy.formation' outcomes
data['ileostomy.formation'] = data['ileostomy.formation'].replace({'New ileostomy':'Ileostomy','Other ileostomy revision':'Ileostomy','Conversion of end ileostomy to loop':'Ileostomy','Ileostomy from prior is unaltered':'Ileostomy'})


In [127]:
#if 'height.unit' is 'in', convert to 'cm' | if 'weight.unit' is 'lb', convert to 'kg'
#get height and weight to the correct units
for x in data.index:
    if data['height.unit'][x] == 'in':
        data['height'][x] = data['height'][x] * 2.54
    if data['weight.unit'][x] == 'lb':
        data['weight'][x] = data['weight'][x] * 0.453592

/tmp/ipykernel_3706035/3933278702.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['height'][x] = data['height'][x] * 2.54


In [128]:
# there is an outcome called 'ibd_ileost' that wasn't addressed in preproc.ipynb
# guess we'll call it ileostomy
data['ileostomy.formation'] = data['ileostomy.formation'].replace({'ibd_ileost':'Ileostomy'})
data['ileostomy.formation'].value_counts()

None         200
Ileostomy     56
Name: ileostomy.formation, dtype: int64

In [129]:
data['colectomy.operative.approach'].value_counts()

MIS                                  47
Open                                 13
MIS, unplanned conversion to open     6
Name: colectomy.operative.approach, dtype: int64

In [130]:
# large amount of NA in the 'colectomy.operative.approach'
data['colectomy.operative.approach'].value_counts(dropna=False)

NaN                                  190
MIS                                   47
Open                                  13
MIS, unplanned conversion to open      6
Name: colectomy.operative.approach, dtype: int64

In [141]:
#renaming columns
data.columns = data.columns.str.replace('surgical.wound.s..closure','surgical.wound(s).closure',regex=False)
data.columns = data.columns.str.replace('sepsis,.sirs.sepsis.septic.shock.>.48h.','sepsis.(sirs/sepsis/septic.shock).(48h)',regex=False)
data.columns = data.columns.str.replace('duration.of.surgical.procedure,.in.minutes.','duration.of.surgical.procedure.(in.minutes)',regex=False)
data.columns = data.columns.str.replace('#.of.postop.transfusion.intraop,.postop,.72h.of.surgery.start.time.','#.of.postop.transfusion.intraop/.postop.(72h.of.surgery.start.time)',regex=False)
data.columns = data.columns.str.replace('x.','#',regex=False)
data.columns = data.columns.str.replace('w.in','w/in',regex=False)
data.columns = data.columns.str.replace('...','.>.',regex=False)
data.columns = data.columns.str.replace('..',',.',regex=False)
data.columns = data.columns.str.replace('procedure.surgery','procedure/surgery',regex=False)
data.columns = data.columns.str.replace('organ.space','organ/space',regex=False)
data.columns = data.columns.str.replace('steroid.immunosuppressant','steroid/immunosuppressant',regex=False)
data.columns = data.columns.str.replace('in.out.patient.status','in/out-patient.status',regex=False)
data.columns = data.columns.str.replace('preop.transfusions,.rbc.w/in.72.hrs.prior.to.surgery.start.time.','preop.transfusions.(rbc.w/in.72.hrs.prior.to.surgery.start.time)',regex=False)
data.columns = data.columns.str.replace('ast.sgot','ast/sgot',regex=False)
data.columns = data.columns.str.replace('#.of.postop.transfusion.intraop..postop..72h.of.surgery.start.time.','#.of.postop.transfusion.intraop/.postop.(72h.of.surgery.start.time)',regex=False)
data.columns = data.columns.str.replace('preop.transfusions..rbc.w/in.72.hrs.prior.to.surgery.start.time.','preop.transfusions.(rbc.w/in.72.hrs.prior.to.surgery.start.time)',regex=False)
data.columns = data.columns.str.replace('#.of.postop.c,.diff','#.of.postop.c..diff',regex=False)
data.columns = data.columns.str.replace('colectomy.non.emergent.indication.icd10.code','colectomy.non-emergent.indication.icd10.code',regex=False)

In [132]:
#list(data)

In [133]:
data_1_4 = pd.read_csv('data/data_clean1.csv',na_values='Unknown')
data_1_4_vars = list(data_1_4)

In [134]:
data_1_4.shape

(1302, 90)

In [135]:
yr_5_vars = list(data)
sum = 0
for i in range(len(data_1_4_vars)):
    if (data_1_4_vars[i] in yr_5_vars):
        #print("IN",data_1_4_vars[i])
        sum+=1
    else:
        print("OUT",data_1_4_vars[i])
print(sum)


OUT >10%.loss.of.body.weight.in.the.6.months.prior.to.surgery
OUT sepsis.(sirs/sepsis/septic.shock).(48h)
OUT duration.of.surgical.procedure.(in.minutes)
OUT #.of.postop.transfusion.intraop/.postop.(72h.of.surgery.start.time)
86


In [136]:
data_1_4['>10%.loss.of.body.weight.in.the.6.months.prior.to.surgery'].value_counts(dropna=False)

NaN    706
No     513
Yes     83
Name: >10%.loss.of.body.weight.in.the.6.months.prior.to.surgery, dtype: int64

In [137]:
data['malnourishment'].value_counts(dropna=False)

NaN    256
Name: malnourishment, dtype: int64

In [138]:
data_1_4.drop(['>10%.loss.of.body.weight.in.the.6.months.prior.to.surgery'],axis=1,inplace=True)
data_1_4_vars = list(data_1_4)

In [142]:
yr_5_vars = list(data)
sum = 0
for i in range(len(data_1_4_vars)):
    if (data_1_4_vars[i] in yr_5_vars):
        #print("IN",data_1_4_vars[i])
        sum+=1
    else:
        print("OUT",data_1_4_vars[i])
print(sum)

89


In [143]:
data_5 = data[data_1_4_vars]

In [144]:
comb_data = pd.concat([data_1_4,data_5],axis=0)
comb_data.shape

(1558, 89)

In [145]:
comb_data.to_csv('data/comb5_data.csv', index=False)